In [1]:
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.models import load_model

IMAGE = 0
MASK = 1

class unet_prj:
    """
    """
    def __init__(self, path, image_info=(256, 256, 1)):
        """
        path - directory that includes img and model file
        image_info - shape img
        """
        self.PATH = path 
        self.IMINFO = image_info
        self.HIST = 0
        self.EPOH = 0
        self.MODEL = 0
    def __get_model(self, pretrained_weights=None):
        """
        This method create model arhitecture
        and return it
        """
        inputs = Input(self.IMINFO)
        conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
        conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
        conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
        conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
        conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
        conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
        drop4 = Dropout(0.5)(conv4)
        pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

        conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
        conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
        drop5 = Dropout(0.5)(conv5)

        up6 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(drop5))
        merge6 = concatenate([drop4, up6], axis=3)
        conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
        conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

        up7 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv6))
        merge7 = concatenate([conv3, up7], axis=3)
        conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
        conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

        up8 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv7))
        merge8 = concatenate([conv2, up8], axis=3)
        conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
        conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

        up9 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv8))
        merge9 = concatenate([conv1, up9], axis=3)
        conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
        conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
        conv9 = Conv2D(2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
        conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

        model = Model(inputs, conv10, name="UNET")
        model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

        return model

    def __loadimg(self):
        x = []
        y = []
        good = os.listdir(self.PATH + "\\data")
        bad = os.listdir(self.PATH + "\\mask")
        for i in good:
            img = cv2.imread(self.PATH + "\\data" + f"\\{i}", 0)
            img = cv2.resize(img, (256, 256))
            img = img / 255.0
            x.append(img)
        for i in bad:
            img = cv2.imread(self.PATH + "\\mask" + f"\\{i}", 0)
            img = cv2.resize(img, (256, 256))
            y.append(img)

        return x, y

    def load_model(self):
        self.MODEL = load_model(self.PATH+'\\save\\Unet.h5')

    def train_model(self, epoh, batchsz, save=True):
        self.EPOH = epoh
        self.MODEL = self.__get_model()
        train_x, train_y = self.__loadimg()
        train_x = np.array(train_x)
        train_y = np.array(train_y)
        self.HIST = self.MODEL.fit(train_x, train_y, epochs=self.EPOH, batch_size=batchsz)

        if (save):
            newpth = self.PATH+"\\save"
            if(os.path.exists(newpth)):
                self.MODEL.save(newpth+"\\Unet.h5", self.HIST)
            else:
                os.mkdir(newpth)
                self.MODEL.save(newpth+"\\Unet.h5", self.HIST)
            print(f"Model save as {newpth}\\Unet.h5")

    def imanalys(self, impath):
        img = cv2.imread(impath, 0)
        img = cv2.resize(img, (256, 256))
        img = img / 255.0
        img = np.array([img])
        out = self.MODEL.predict(img)
        out = out.reshape((256, 256))
        out = cv2.cvtColor(out, cv2.COLOR_GRAY2RGB)
        cv2.imshow("Resault", out)
        cv2.waitKey(0)

        return out

    def imcheck(self, cvimage, mode=IMAGE):
        """
        This function check model result
        """
        image = cvimage
        image = cv2.resize(image, (1024, 1024))
        if(mode == 1):
            for i in range(np.shape(image)[0]):
                for j in range(np.shape(image)[1]):
                    # base on model result draw good and bad cells 
                    if (image[i][j][0] == 1):
                        image[i][j] = [255, 0, 0]
                    elif (image[i][j][0] > 0.5):
                        image[i][j] = [0, 0, 255]
        cv2.imshow("Check image", image)
        cv2.waitKey()

    def showgraph(self):
        N = np.arange(0, self.EPOH)
        plt.style.use("ggplot")
        plt.figure()
        plt.plot(N, self.HIST.history["loss"], label="train_loss")
        plt.plot(N, self.HIST.history["accuracy"], label="train_acc")
        plt.title("Training Loss and Accuracy")
        plt.xlabel("Epoch #")
        plt.ylabel("Loss/Accuracy")
        plt.legend()
        plt.show()

In [2]:
x = unet_prj(path="E:\\shlak\\maskBlodDS")
#x.load_model()
x.train_model(epoh=100, batchsz=5)
x.showgraph()
img = x.imanalys(impath="E:\\shlak\\maskBlodDS\\data\\44.png")
x.imcheck(img, MASK)

1/1 [==============================] - 4s 4s/step
